In [159]:
# load the libraries
import medspacy
from medspacy.ner import TargetRule
from medspacy.visualization import visualize_ent

In [160]:
import pandas as pd

In [161]:
# load the model and the piples
nlp = medspacy.load()
target_matcher = nlp.get_pipe("medspacy_target_matcher")

In [162]:
# load the dataset
df = pd.read_table("./input/test.dat", header=None)

# get the first 3 lines
partial_df = df[0:3]

In [ ]:
# load encoding
from spacy.tokens import Span

Span.set_extension("icd10", default="", force=True)

In [163]:
# define some custom rules
target_rules = [
    TargetRule("Diabetes", category="PROBLEM",
               pattern=[
                   {"LOWER": "diabetes"},
                   {"LOWER": {"IN": ["diabetes", "diabetic"]}},
               ],
               attributes={"icd10": "E11.9"}),
    TargetRule("Bell's palsy", category="PROBLEM",
               pattern=[
                   {"LOWER": "bell", "OP": "?"},
                   {"LOWER": "palsy"}
               ],
               attributes={"icd10": "G80.9"}),
    TargetRule("Hypertension", category="PROBLEM",
               pattern=[
                   {"LOWER": "high"},
                   {"LOWER": {"IN": ["blood", "bp"]}},
                   {"LOWER": "pressure", "OP": "?"}
               ],
               attributes={"icd10": "I10"}),
    TargetRule("Heart Disease", category="PROBLEM",
               pattern=[
                   {"LOWER": "heart"},
                   {"LOWER": {"IN": ["disease", "attack"]}},
               ],
               attributes={"icd10": "I51.9"}),
    TargetRule("Buccal Abscess", category="PROBLEM",
               pattern=[
                   {"LOWER": "buccal"},
                   {"LOWER": {"IN": ["abscess", "abscesses"]}},
               ],
               attributes={"icd10": "K12. 2"}),
    TargetRule("Intracranial fibromatosis", category="PROBLEM",
               pattern=[
                   {"LOWER": "intracranial fibromatosis"},
                   {"LOWER": {"IN": ["intracranial", "cranial", "fibromatosis"]}},
               ],
               attributes={"icd10": "D36.10"}),
    TargetRule("Idiopathic facial paralysis", category="PROBLEM",
               pattern=[
                   {"LOWER": "idiopathic", "OP": "?"},
                   {"LOWER": "facial", "OP": "?"},
                   {"LOWER": {"IN": ["paralysis", "palsy"]}},
               ],
               attributes={"icd10": "D36.10"}),
    TargetRule("corticosteroids", category="MEDICATION",
               pattern=[
                   {"LOWER": "oral"},
                   {"LOWER": "corticosteroids"}
               ],
               attributes={"icd10": "D36.10"}),
    TargetRule("acute unilateral facial weakness", category="PROBLEM",
               pattern=[
                   {"LOWER": {"IN": ["acute", "unilateral", "bilateral"]}, "OP": "?"},
                   {"LOWER": "facial"},
                   {"LOWER": "weakness"}
               ],
               attributes={"icd10": "R29.810"}),
    TargetRule("stroke", category="PROBLEM",
               pattern=[
                   {"LOWER": "stroke"}
               ],
               attributes={"icd10": "I63.9"}),
    TargetRule("eye protection", category="TREATMENT",
               pattern=[
                   {"LOWER": "eye"},
                   {"LOWER": "protection"}
               ]),
    TargetRule("limbal dermoid", category="PROBLEM",
               pattern=[
                   {"LOWER": {"IN": ["unilateral", "bilateral"]}, "OP": "?"},
                   {"LOWER": "epilbular", "OP": "?"},
                   {"LOWER": "limbal"},
                   {"LOWER": {"IN": ["dermoid", "dermoids"]}}
               ]),
    TargetRule("visual acuity", category="PROBLEM",
               pattern=[
                   {"LOWER": "visual", "OP": "?"},
                   {"LOWER": "acuity"},
               ]),
    TargetRule("astigmatism", category="PROBLEM",
               pattern=[
                   {"LOWER": "astigmatism"},
               ]),
    TargetRule("epithelial defects", category="PROBLEM",
               pattern=[
                   {"LOWER": "corneal", "OP": "?"},
                   {"LOWER": "astigmatism"},
                   {"LOWER": "defects"},
               ]),
    TargetRule("corneal vascularization", category="PROBLEM",
               pattern=[
                   {"LOWER": "corneal", "OP": "?"},
                   {"LOWER": "vascularization"}
               ]),
    TargetRule("topical lubrication", category="TREATMENT",
               pattern=[
                   {"LOWER": "topical", "OP": "?"},
                   {"LOWER": "lubrication"}
               ]),
    TargetRule("excision", category="TREATMENT",
               pattern=[
                   {"LOWER": "excision"}
               ])
]

In [164]:
# add custom rules to pipelines
target_matcher.add(target_rules)

In [165]:
# test the pipeline on the dataset
for index, row in partial_df.iterrows():
    doc = nlp(row[0])
    print("Sentence:", doc.text)
    # print(nlp.pipe_names)

    for ent in doc.ents:
        print(ent, ent._.is_negated, ent._.is_family)

    visualize_ent(doc)

    print("###################")

Sentence: Excision of limbal dermoids. We reviewed the clinical files of 10 patients who had undergone excision of unilateral epibulbar limbal dermoids. These complications do not outweigh the cosmetic and visual benefits of dermoid excision in selected patients. Treatment of limbal dermoids may consist of periodic removal of irritating cilia, topical lubrication to prevent foreign body sensation, or excision of the lesion if it is causing significant cosmetic disfigurement or interfering with vision. Treatment of limbal dermoids may consist of periodic removal of irritating cilia, topical lubrication to prevent foreign body sensation, or excision of the lesion if it is causing significant cosmetic disfigurement or interfering with vision.
Excision False False
limbal dermoids False False
excision False False
limbal dermoids False False
excision True False
limbal dermoids False False
topical lubrication False False
excision False False
limbal dermoids False False
topical lubrication Fal

###################
Sentence: Past medical history: type II Diabetes Mellitus. Bell's palsy. There's no evidence of stroke. A diagnosis of exclusion. In cases of acute unilateral facial weakness, a careful and systematic evaluation is necessary to identify the cause. Idiopathic facial paralysis (Bell's palsy) is a diagnosis of exclusion. It is also the most common cause of unilateral facial weakness seen by primary care physicians. The most important aspect of initial treatment is eye protection. Administration of systemic oral corticosteroids may lessen severity and duration of symptoms.
palsy False False
stroke True False
unilateral facial weakness False False
Idiopathic facial paralysis False False
palsy False False
unilateral facial weakness False False
eye protection False False
oral corticosteroids False False


###################
Sentence: Retained endobronchial foreign body removal facilitated by steroid therapy of an obstructing, inflammatory polyp. Oral and topical steroids were used to induce regression in an inflammatory, obstructing endobronchial polyp caused by a retained foreign body. The FB (a peanut half), which had been present for over six months, was then able to be easily and bloodlessly retrieved with fiberoptic bronchoscopy. 


###################
